In [ ]:
!pip install --upgrade llvmlite
!pip install --upgrade git+https://github.com/scikit-learn-contrib/hdbscan.git#egg=hdbscan
!pip install bertopic
# !pip install bertopic[flair]
# !pip install bertopic[gensim]
# !pip install bertopic[spacy]
# !pip install bertopic[use]
!pip install git+https://github.com/MartinoMensio/spacy-universal-sentence-encoder.git

In [ ]:
import pandas as pd

from bertopic import BERTopic
import pandas as pd
import numpy as np
import multiprocessing
import time
import csv
import io

multiprocessing.cpu_count()


from google.colab import drive
#import os
!pip install pandas==1.5.3
#!pip install pandas==1.3.5
import pandas as pd

drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/MyDrive/MITB_AI/Text analytics dataset

In [ ]:
%ls

In [ ]:
X_train = pd.read_pickle("X_Train.pkl")

from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # min occurrences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=False,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

df_filtered_lemmatized = X_train["text_lemmatized"].tolist()

stop_list = ("suicidal", "suicide", "aah", "fuck")
df_filtered_lemmatized_removeSuicide = [[w for w in doc if w not in stop_list] for doc in df_filtered_lemmatized]
df_filtered_lemmatized_joined = [' '.join(x) for x in df_filtered_lemmatized_removeSuicide]  # joined to fit CountVectorizer
train1_vecs = vectorizer.fit_transform(df_filtered_lemmatized_joined) #may be needed for bertopic
feature_names = vectorizer.get_feature_names_out()
# display the first 20 tokens
feature_names[:20]

In [ ]:
print(train1_vecs.shape)

In [ ]:
start_time = time.time()

model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(df_filtered_lemmatized_joined)

print('Total time taken (mins): ', int((time.time()-start_time)/60))

In [ ]:
model.get_topic_info().head()

In [ ]:
import joblib
joblib.dump(model, 'BERTopic_X_Train.jl')

In [ ]:
import joblib
model = joblib.load('BERTopic_X_Train.jl')

In [ ]:
model.reduce_topics(df_filtered_lemmatized_joined, nr_topics=30)

In [ ]:
topic_keywords = model.get_topics()

df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

In [ ]:
df_topic_keywords.to_csv('BERTopic_X_Train_TopicsOutput.csv')